<a href="https://colab.research.google.com/github/abdullahmujahidali/Vet-LangGraph/blob/main/VetAI_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install -q langchain langgraph langchain_openai openai python-dotenv

In [24]:
import os
from typing import List, Dict, Tuple, Annotated, TypedDict, Union, Any
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
import json

from google.colab import userdata

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0
)

In [25]:
class VetState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]
    current_input: str
    medical_records: Dict[str, Any]
    owner_history: Dict[str, Any]
    diagnoses: Dict[str, Any]
    results: Dict[str, Any]

def create_data_processing_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary data processing agent responsible for initial data extraction and organization.

KEY RESPONSIBILITIES:
1. File Processing:
   - Process veterinary records, lab reports, and supporting documents
   - Extract and structure content based on document type
   - Apply standardized formatting for consistent output

2. Content Organization:
   - Patient Information (Name, Species, Breed, Age, Medical Record Number)
   - Clinical History & Physical Exam findings
   - SOAP Notes and Progress Reports
   - Medications & Treatments (with precise dosing)
   - Laboratory Results (with reference ranges)
   - Diagnostic Imaging Reports
   - Doctor's Recommendations

3. Data Standardization:
   - Tag and categorize information for easy retrieval
   - Structure data in consistent format
   - Flag any missing or unclear information
   - Maintain medical terminology standards

4. Privacy & Security:
   - Identify and flag sensitive information
   - Ensure data security compliance
   - Note any areas requiring manual review"""),
        ("human", "Process and structure the following veterinary input: {input}")
    ])

    def process_data(state: VetState):
        messages = prompt.format_messages(input=state["current_input"])
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["DataProcessor"] = response.content
        return new_state

    return process_data

def create_history_analysis_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary history analysis agent specializing in comprehensive medical history evaluation.

KEY RESPONSIBILITIES:
1. Medical History Analysis:
   - Extract all past and current medical conditions
   - Categorize by body system (e.g., gastrointestinal, dermatologic)
   - Document onset dates, severity, and progression
   - Identify primary presenting complaints
   - Distinguish between acute, chronic, recurrent conditions

2. Treatment History Documentation:
   - Track all medications with exact details:
     * Drug names (brand and generic)
     * Formulations and concentrations
     * Dosing information (amount, frequency, duration)
     * Calculate mg/kg when weight is available
     * Document response to therapy
     * Note any adverse reactions

3. Problem Categorization:
   - Active vs. Historical Problems
   - Related vs. Separate Issues
   - Primary vs. Secondary Conditions
   - Track problem progression and status

4. Owner Input Analysis:
   - Document owner-reported symptoms
   - Note owner concerns and observations
   - Record compliance history
   - Identify practical limitations"""),
        ("human", "Based on this processed data, provide a comprehensive medical history analysis: {processed_data}")
    ])

    def analyze_history(state: VetState):
        processed_data = state["results"]["DataProcessor"]
        messages = prompt.format_messages(processed_data=processed_data)
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["HistoryAnalyzer"] = response.content
        return new_state

    return analyze_history

def create_clinical_analysis_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary clinical record analyst responsible for comprehensive medical record review and synthesis.

KEY RESPONSIBILITIES:
1. Record Analysis:
   - Review and synthesize all medical documentation
   - Track patient details and trends
   - Analyze laboratory results and imaging
   - Document owner preferences and compliance

2. Tracking Requirements:
   - Weight and BCS trends
   - Laboratory value trends
   - Response to treatments
   - Disease progression patterns
   - Dietary history and responses

3. Clinical Pattern Recognition:
   - Identify related symptoms and conditions
   - Track disease progression
   - Note treatment responses
   - Flag unusual presentations
   - Document complications

4. Integration Analysis:
   - Correlate findings across records
   - Compare subjective and objective data
   - Evaluate treatment effectiveness
   - Identify gaps in documentation"""),
        ("human", "Based on this history analysis, provide a detailed clinical analysis: {history_analysis}")
    ])

    def analyze_clinical_records(state: VetState):
        history_analysis = state["results"]["HistoryAnalyzer"]
        messages = prompt.format_messages(history_analysis=history_analysis)
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["ClinicalAnalyzer"] = response.content
        return new_state

    return analyze_clinical_records

def create_diagnostic_agent():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a veterinary diagnostic intelligence agent responsible for analyzing and categorizing medical problems with confidence levels.

KEY RESPONSIBILITIES:
1. Problem Classification:
   - Categorize diagnoses by type:
     * Definitive diagnoses
     * Differential diagnoses
     * Symptom-based issues
     * Laboratory abnormalities
   - Assign confidence levels:
     * Confirmed (diagnostic evidence)
     * Presumptive (clinical suspicion)
     * Owner-reported only

2. Diagnostic Analysis:
   - Cross-reference symptoms with conditions
   - Evaluate diagnostic test results
   - Consider breed-specific conditions
   - Factor in age-related concerns

3. Pattern Recognition:
   - Identify disease patterns
   - Note unusual presentations
   - Flag inconsistencies
   - Track response patterns

4. Treatment Impact Assessment:
   - Evaluate therapeutic responses
   - Document treatment failures
   - Note adverse reactions
   - Track condition progression"""),
        ("human", "Based on this clinical analysis, provide diagnostic insights and recommendations: {clinical_analysis}")
    ])

    def diagnose(state: VetState):
        clinical_analysis = state["results"]["ClinicalAnalyzer"]
        messages = prompt.format_messages(clinical_analysis=clinical_analysis)
        response = llm.invoke(messages)

        new_state = state.copy()
        new_state["messages"].append(response)
        new_state["results"]["Diagnostics"] = response.content
        return new_state

    return diagnose



In [26]:
data_processor = create_data_processing_agent()
history_analyzer = create_history_analysis_agent()
clinical_analyzer = create_clinical_analysis_agent()
diagnostic_agent = create_diagnostic_agent()

workflow = StateGraph(VetState)

workflow.add_node("process_data", data_processor)
workflow.add_node("analyze_history", history_analyzer)
workflow.add_node("analyze_clinical", clinical_analyzer)
workflow.add_node("diagnose", diagnostic_agent)

workflow.set_entry_point("process_data")

workflow.add_edge("process_data", "analyze_history")
workflow.add_edge("analyze_history", "analyze_clinical")
workflow.add_edge("analyze_clinical", "diagnose")
workflow.add_edge("diagnose", END)

app = workflow.compile()


In [27]:
test_case_1 = {
    "messages": [],
    "current_input": """
    Patient Information:
    Name: Luna
    Species: Feline
    Breed: Domestic Shorthair
    Age: 13 years
    Weight: 4.2 kg (Previous: 4.8 kg 3 months ago)
    Sex: Female Spayed

    History:
    Presenting Complaints:
    - Weight loss despite good appetite
    - Increased thirst and urination
    - Occasional vomiting (2-3 times per week)
    - Vocal at night

    Previous Medical History:
    - Dental cleaning and extraction (2 teeth) in 2023
    - Historical UTI treated in 2022

    Recent Diagnostics:
    - CBC/Chemistry (1 week ago):
      * T4: 62 nmol/L (15-50)
      * ALT: 98 U/L (10-90)
      * BUN: 35 mg/dL (16-36)
      * Creatinine: 1.8 mg/dL (0.8-2.4)
      * Phosphorus: 4.5 mg/dL (3.1-7.5)

    Current Medications:
    - Methimazole 2.5mg BID (started 3 days ago)

    Owner Notes:
    - Lives indoor only
    - Eating well but losing weight
    - More active at night
    - Financial constraints noted
    - Prefers pill form medications (easy to administer)
    """,
    "medical_records": {},
    "owner_history": {},
    "diagnoses": {},
    "results": {}
}


result = app.invoke(test_case_1)

print("\nData Processing Results:")
print(result["results"]["DataProcessor"])
print("\nHistory Analysis:")
print(result["results"]["HistoryAnalyzer"])
print("\nClinical Analysis:")
print(result["results"]["ClinicalAnalyzer"])
print("\nDiagnostic Assessment:")
print(result["results"]["Diagnostics"])


Data Processing Results:
**Patient Information:**
- Name: Luna
- Species: Feline
- Breed: Domestic Shorthair
- Age: 13 years
- Weight: 4.2 kg (Previous: 4.8 kg 3 months ago)
- Sex: Female Spayed

**History:**
**Presenting Complaints:**
- Weight loss despite good appetite
- Increased thirst and urination
- Occasional vomiting (2-3 times per week)
- Vocal at night

**Previous Medical History:**
- Dental cleaning and extraction (2 teeth) in 2023
- Historical UTI treated in 2022

**Recent Diagnostics:**
- CBC/Chemistry (1 week ago):
  - T4: 62 nmol/L (15-50)
  - ALT: 98 U/L (10-90)
  - BUN: 35 mg/dL (16-36)
  - Creatinine: 1.8 mg/dL (0.8-2.4)
  - Phosphorus: 4.5 mg/dL (3.1-7.5)

**Current Medications:**
- Methimazole 2.5mg BID (started 3 days ago)

**Owner Notes:**
- Lives indoor only
- Eating well but losing weight
- More active at night
- Financial constraints noted
- Prefers pill form medications (easy to administer)

History Analysis:
**Comprehensive Medical History Analysis for Luna: